<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/nemo_ft_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!apt-get update && apt-get install -y graphviz
!pip install ipywidgets
!pip install --upgrade setuptools wheel

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,892 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,711 kB]
Fetched 13.0 MB in 2s (5,4

In [7]:
!pip install nemo_toolkit[all]==2.6.1 -q

In [8]:
!pip cache purge
!pip install --no-build-isolation transformer-engine[pytorch] -q
!pip install nemo_run opendatasets pandas bitsandbytes accelerate -q
!pip install --upgrade transformers -q

Files removed: 1323


KeyboardInterrupt: 

In [ ]:
!pip install "numpy<2.0" --force-reinstall

In [9]:
from huggingface_hub import login
from google.colab import userdata

# Login to Hugging Face
login(token=userdata.get("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [1]:
from pathlib import Path

import nemo_run as run
from nemo import lightning as nl
from nemo.collections import llm
from nemo.collections.llm.recipes.precision.mixed_precision import bf16_mixed

/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
[NeMo W 2026-02-07 11:32:30 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/transformer_engine/__init__.py:59: RuntimeWarning: Detected a Jax installation but could not find the shared object file for the Transformer Engine Jax extension library. If this is not intentional, please reinstall Transformer Engine with `pip install transformer_engine[jax]` or build from source with `NVTE_FRAMEWORK=jax`.
      warnings.warn(
    
[NeMo W 2026-02-07 11:32:48 nemo_logging:405] The deploy module could not be imported: cannot import name 'deploy' from 'nemo.collections.llm.api' (/usr/local/lib/python3.12/dist-packages/nemo/collections/llm/api.py)
[NeMo W 2026-02-07 11:32:48 

In [2]:
import os
import nemo_run as run
from nemo.collections import llm
import nemo as ne
from nemo import lightning as nl
import transformer_engine as te
import transformers as tr


print(f"Nemo version: {ne.__version__}")
print(f"NeMo RUN version: {run.__version__}")
print(f"Transformer Engine version: {te.__version__}")
print(f"Transformers version: {tr.__version__}")

Nemo version: 2.6.1
NeMo RUN version: 0.7.0
Transformer Engine version: 2.11.0
Transformers version: 4.53.3


In [ ]:
!rm -rf /content/*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -rf /content/*

In [ ]:
import os
import json
import torch
import tarfile
import dataclasses
from pathlib import Path
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. AUTHENTICATION & DIRECTORY SETUP
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
MODEL_SOURCE = "meta-llama/Meta-Llama-3-8B"
COLAB_BASE = "/content/nemo_llama3_final"
NEMO_FILE = f"{COLAB_BASE}/llama3_8b_fixed.nemo"
WORKSPACE = f"{COLAB_BASE}/workspace"

os.makedirs(WORKSPACE, exist_ok=True)

# 2. ARCHITECTURE CONFIGURATION
from nemo.collections.llm.gpt.model.llama import Llama3Config8B
config = Llama3Config8B(seq_length=512, bf16=True)

def clean_nemo_config(cfg):
    """Converts dataclass to JSON-serializable dict."""
    c = dataclasses.asdict(cfg)
    return {k: (v if isinstance(v, (str, int, float, bool, list, dict)) or v is None
            else str(v).split('.')[-1]) for k, v in c.items()}

# 3. ROBUST .NEMO CREATION
if not os.path.exists(NEMO_FILE):
    print("🚀 Building .nemo file with standard tokenizer mapping...")

    # Save Weights
    hf_model = AutoModelForCausalLM.from_pretrained(MODEL_SOURCE, torch_dtype=torch.bfloat16, device_map="cpu")
    weights_path = os.path.join(WORKSPACE, "weights")
    os.makedirs(weights_path, exist_ok=True)
    torch.save(hf_model.state_dict(), os.path.join(weights_path, "common.pt"))

    # Save Tokenizer to a dedicated sub-folder
    tokenizer = AutoTokenizer.from_pretrained(MODEL_SOURCE)
    tokenizer_dir = os.path.join(WORKSPACE, "tokenizer")
    os.makedirs(tokenizer_dir, exist_ok=True)
    tokenizer.save_pretrained(tokenizer_dir)

    # Create NeMo context metadata (io.json)
    # NOTE: We use the HuggingFace AutoTokenizer target directly for NeMo 2.6 compatibility.
    io_json_path = os.path.join(WORKSPACE, "context", "io.json")
    os.makedirs(os.path.dirname(io_json_path), exist_ok=True)
    with open(io_json_path, 'w') as f:
        json.dump({
            "_target_": "nemo.collections.llm.gpt.model.llama.LlamaModel",
            "config": clean_nemo_config(config),
            "tokenizer": {
                "_target_": "nemo.collections.common.tokenizers.huggingface.AutoTokenizer",
                "pretrained_model_name_or_path": "model/tokenizer" # Path relative to .nemo root
            }
        }, f, indent=2)

    # Package into .nemo
    with tarfile.open(NEMO_FILE, "w:gz") as tar:
        for root, _, files in os.walk(WORKSPACE):
            for file in files:
                full_path = os.path.join(root, file)
                # Map the local workspace contents into the "model/" prefix inside the archive
                tar.add(full_path, arcname=os.path.join("model", os.path.relpath(full_path, WORKSPACE)))
    print(f"✅ Created {NEMO_FILE}")

# 4. EXECUTE TRAINING WITH FIXED RESTORATION
import nemo_run as run
from nemo import lightning as nl
from nemo.collections import llm
from nemo.collections.llm.recipes.precision.mixed_precision import bf16_mixed

print("\n>>> Starting LoRA Fine-Tuning with Validated Tokenizer...")
recipe = run.Partial(
    llm.finetune,
    model=run.Config(llm.LlamaModel, config=run.Config(llm.Llama3Config8B)),
    data=run.Config(llm.SquadDataModule, seq_length=512, micro_batch_size=1, global_batch_size=4),
    trainer=run.Config(
        nl.Trainer, devices=1, max_steps=20, accelerator="gpu",
        strategy=run.Config(nl.MegatronStrategy, tensor_model_parallel_size=1),
        plugins=bf16_mixed()
    ),
    peft=run.Config(llm.peft.LoRA, target_modules=['linear_qkv']),
    resume=run.Config(
        nl.AutoResume,
        restore_config=run.Config(nl.RestoreConfig, path=NEMO_FILE),
        resume_if_exists=True
    )
)

run.run(recipe, executor=run.LocalExecutor(ntasks_per_node=1, launcher="torchrun"))

## CASE 2

In [ ]:
import os
import json
import torch
import tarfile
import dataclasses
from pathlib import Path
from google.colab import userdata
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. SETUP ENVIRONMENT & AUTHENTICATION
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
MODEL_SOURCE = "meta-llama/Meta-Llama-3-8B"
COLAB_BASE = "/content/nemo_llama3_manual"
NEMO_FILE = f"{COLAB_BASE}/llama3_8b_manual.nemo"
WORKSPACE = f"{COLAB_BASE}/workspace"
TRAIN_DATA = f"{COLAB_BASE}/toy_train.jsonl"
os.makedirs(WORKSPACE, exist_ok=True)

# --- STEP 0: CREATE DATASET (FIX) ---
# This ensures TRAIN_DATA exists before the loop starts
if not os.path.exists(TRAIN_DATA):
    print(f"📊 Creating {TRAIN_DATA}...")
    samples = [
        {"input": "Context: NeMo is a toolkit. Question: What is NeMo? Answer: A toolkit"},
        {"input": "Context: Llama 3 is a model. Question: What is Llama 3? Answer: A model"}
    ]
    with open(TRAIN_DATA, "w") as f:
        for s in samples:
            f.write(json.dumps(s) + "\n")

# 2. ARCHITECTURE CONFIGURATION
from nemo.collections.llm.gpt.model.llama import Llama3Config8B
config = Llama3Config8B(seq_length=512, bf16=True)

# 3. MANUAL .NEMO CREATION (Reference Step 3)
if not os.path.exists(NEMO_FILE):
    print(f"🚀 Building {NEMO_FILE}...")
    hf_model = AutoModelForCausalLM.from_pretrained(MODEL_SOURCE, torch_dtype=torch.bfloat16, device_map="cpu")
    weights_path = os.path.join(WORKSPACE, "weights")
    os.makedirs(weights_path, exist_ok=True)
    torch.save(hf_model.state_dict(), os.path.join(weights_path, "common.pt"))

    def clean_nemo_config(cfg):
        c = dataclasses.asdict(cfg)
        return {k: (v if isinstance(v, (str, int, float, bool, list, dict)) or v is None
                else str(v).split('.')[-1]) for k, v in c.items()}

    io_json_path = os.path.join(WORKSPACE, "context", "io.json")
    os.makedirs(os.path.dirname(io_json_path), exist_ok=True)
    with open(io_json_path, 'w') as f:
        json.dump({"_target_": "nemo.collections.llm.gpt.model.llama.LlamaModel", "config": clean_nemo_config(config)}, f, indent=2)

    with tarfile.open(NEMO_FILE, "w:gz") as tar:
        for root, _, files in os.walk(WORKSPACE):
            for file in files:
                full_path = os.path.join(root, file)
                tar.add(full_path, arcname=os.path.join("model", os.path.relpath(full_path, WORKSPACE)))

In [ ]:
import os
import json
import torch
import tarfile
import dataclasses
from pathlib import Path
from google.colab import userdata
from transformers import AutoModelForCausalLM, AutoTokenizer as HFAutoTokenizer

# Correct NeMo Imports
from nemo.collections.common.tokenizers.huggingface import AutoTokenizer as NeMoAutoTokenizer
from nemo.collections import llm

# 1. SETUP ENVIRONMENT & PATHS
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
MODEL_SOURCE = "meta-llama/Meta-Llama-3-8B"
COLAB_BASE = "/content/nemo_llama3_manual"
NEMO_FILE = f"{COLAB_BASE}/llama3_8b_manual.nemo"
WORKSPACE = f"{COLAB_BASE}/workspace"
TRAIN_DATA = f"{COLAB_BASE}/toy_train.jsonl"
os.makedirs(WORKSPACE, exist_ok=True)

# 2. ARCHITECTURE CONFIGURATION
from nemo.collections.llm.gpt.model.llama import Llama3Config8B
config = Llama3Config8B(seq_length=512, bf16=True)

# 3. CONDITIONAL .NEMO CREATION
if not os.path.exists(NEMO_FILE):
    print(f"🚀 {NEMO_FILE} not found. Creating new .nemo file...")

    # Create Toy Data
    samples = [{"input": "Context: NeMo is a toolkit. Question: What is NeMo? Answer: A toolkit"}]
    with open(TRAIN_DATA, "w") as f:
        for s in samples:
            f.write(json.dumps(s) + "\n")

    # Download HF Model weights
    hf_model = AutoModelForCausalLM.from_pretrained(MODEL_SOURCE, torch_dtype=torch.bfloat16, device_map="cpu")
    weights_path = os.path.join(WORKSPACE, "weights")
    os.makedirs(weights_path, exist_ok=True)
    torch.save(hf_model.state_dict(), os.path.join(weights_path, "common.pt"))

    def clean_nemo_config(cfg):
        c = dataclasses.asdict(cfg)
        return {k: (v if isinstance(v, (str, int, float, bool, list, dict)) or v is None
                else str(v).split('.')[-1]) for k, v in c.items()}

    # Save Metadata
    io_json_path = os.path.join(WORKSPACE, "context", "io.json")
    os.makedirs(os.path.dirname(io_json_path), exist_ok=True)
    with open(io_json_path, 'w') as f:
        json.dump({
            "model": {
                "_target_": "nemo.collections.llm.gpt.model.llama.LlamaModel",
                "config": clean_nemo_config(config),
                # Note: 'pretrained_model_name' is the API standard for the .nemo context as well
                "tokenizer": {"_target_": "nemo.collections.common.tokenizers.huggingface.AutoTokenizer", "pretrained_model_name": MODEL_SOURCE}
            }
        }, f, indent=2)

    # Package Workspace
    with tarfile.open(NEMO_FILE, "w:gz") as tar:
        for root, _, files in os.walk(WORKSPACE):
            for file in files:
                full_path = os.path.join(root, file)
                tar.add(full_path, arcname=os.path.join("model", os.path.relpath(full_path, WORKSPACE)))
    print(f"✅ Created {NEMO_FILE}")
else:
    print(f"✅ {NEMO_FILE} exists. Skipping to STEP 4.")

# 4. LOAD THE .NEMO FILE AND INITIALIZE (Reference Step 4 & 5)
print("\n🔍 Loading weights and initializing working model...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Extract weights from the .nemo file
with tarfile.open(NEMO_FILE, "r:gz") as tar:
    member = next(m for m in tar.getmembers() if "common.pt" in m.name)
    weights_file = tar.extractfile(member)
    state_dict = torch.load(weights_file, map_location='cpu')

# --- FIX: NeMo API uses 'pretrained_model_name' ---
nemo_tokenizer = NeMoAutoTokenizer(pretrained_model_name=MODEL_SOURCE)
model = llm.LlamaModel(config=config, tokenizer=nemo_tokenizer)

# Build internal layers and load weights
model.configure_model()
model.load_state_dict(state_dict, strict=False)
model.to(device)

# 5. MANUAL TRAINING LOOP (Reference Step 7)
print("🔥 Starting Manual Training Loop...")
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)

# Setup Data
hf_tokenizer = HFAutoTokenizer.from_pretrained(MODEL_SOURCE)
hf_tokenizer.pad_token = hf_tokenizer.eos_token

class ManualDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, tokenizer):
        self.tokenizer = tokenizer
        if os.path.exists(data_path):
            self.samples = [json.loads(line)["input"] for line in open(data_path, 'r')]
        else:
            self.samples = ["Context: Sample. Question: Test? Answer: Yes"]

    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        tokens = self.tokenizer(self.samples[idx], truncation=True, max_length=512, padding="max_length", return_tensors="pt")
        return tokens["input_ids"].squeeze(), tokens["input_ids"].squeeze()

dataloader = torch.utils.data.DataLoader(ManualDataset(TRAIN_DATA, hf_tokenizer), batch_size=1)

for step, (ids, labels) in enumerate(dataloader):
    if step >= 20: break
    ids, labels = ids.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(input_ids=ids, labels=labels)

    # NeMo models return loss directly in training mode
    loss = outputs if isinstance(outputs, torch.Tensor) else outputs.get('loss')

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()

    if step % 5 == 0:
        print(f"Step {step}: Loss = {loss.item():.4f}")

print(f"✅ Training complete. Model is ready.")

## FINAL

In [ ]:
import os
import json
import torch
import tarfile
import dataclasses
import re
import string
import socket
from collections import Counter
from pathlib import Path
from google.colab import userdata
from transformers import AutoModelForCausalLM, AutoTokenizer as HFAutoTokenizer
from rouge_score import rouge_scorer

# NeMo & Megatron Core Imports
from nemo.collections.common.tokenizers.huggingface import AutoTokenizer as NeMoAutoTokenizer
from nemo.collections import llm

# STABLE INITIALIZATION PATHS
try:
    from megatron.core import parallel_state
    from megatron.core.parallel_state import initialize_model_parallel
    # FIX: Import the RNG tracker initializer
    from megatron.core.tensor_parallel import model_parallel_cuda_manual_seed
except ImportError:
    from nemo.utils import get_rank

# 1. UTILITY: FIND AVAILABLE PORT
def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        return s.getsockname()[1]

# 2. SETUP ENVIRONMENT & PATHS
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
MODEL_SOURCE = "meta-llama/Meta-Llama-3-8B"
COLAB_BASE = "/content/nemo_llama3_manual"
NEMO_FILE = f"{COLAB_BASE}/llama3_8b_manual.nemo"
WORKSPACE = f"{COLAB_BASE}/workspace"
TRAIN_DATA = f"{COLAB_BASE}/toy_train.jsonl"
os.makedirs(WORKSPACE, exist_ok=True)

# 3. METRIC CALCULATION LOGIC (Directly from peft_metric_calc.py)
def normalize_answer(s):
    def remove_articles(text): return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text): return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    return white_space_fix(remove_articles(remove_punc(s.lower())))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0: return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    return (2 * precision * recall) / (precision + recall)

def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)

def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    if not isinstance(ground_truths, list): ground_truths = [ground_truths]
    return max([metric_fn(prediction, gt) for gt in ground_truths])

# 4. INITIALIZE DISTRIBUTED CONTEXT & RNG
if not torch.distributed.is_initialized():
    os.environ["MASTER_ADDR"] = "127.0.0.1"
    os.environ["MASTER_PORT"] = str(find_free_port())
    torch.distributed.init_process_group(
        backend="nccl" if torch.cuda.is_available() else "gloo",
        rank=0,
        world_size=1
    )

if not parallel_state.model_parallel_is_initialized():
    initialize_model_parallel(
        tensor_model_parallel_size=1,
        pipeline_model_parallel_size=1,
        virtual_pipeline_model_parallel_size=None,
    )
    # CRITICAL FIX: Initialize the CUDA RNG tracker for model parallelism
    # This prevents the "cuda rng state model-parallel-rng is not added" error
    model_parallel_cuda_manual_seed(42)

# 5. ARCHITECTURE CONFIGURATION
from nemo.collections.llm.gpt.model.llama import Llama3Config8B
config = Llama3Config8B(seq_length=512, bf16=True)

# 6. RESTORED .NEMO CREATION BLOCK
if not os.path.exists(NEMO_FILE):
    print(f"🚀 {NEMO_FILE} not found. Creating new .nemo file...")
    samples = [{"input": "Context: NeMo is a toolkit. Question: What is NeMo? Answer: A toolkit", "label": "A toolkit"}]
    with open(TRAIN_DATA, "w") as f:
        for s in samples: f.write(json.dumps(s) + "\n")

    hf_model = AutoModelForCausalLM.from_pretrained(MODEL_SOURCE, torch_dtype=torch.bfloat16, device_map="cpu")
    weights_path = os.path.join(WORKSPACE, "weights")
    os.makedirs(weights_path, exist_ok=True)
    torch.save(hf_model.state_dict(), os.path.join(weights_path, "common.pt"))

    def clean_nemo_config(cfg):
        c = dataclasses.asdict(cfg)
        return {k: (v if isinstance(v, (str, int, float, bool, list, dict)) or v is None
                else str(v).split('.')[-1]) for k, v in c.items()}

    io_json_path = os.path.join(WORKSPACE, "context", "io.json")
    os.makedirs(os.path.dirname(io_json_path), exist_ok=True)
    with open(io_json_path, 'w') as f:
        json.dump({"model": {"_target_": "nemo.collections.llm.gpt.model.llama.LlamaModel",
                             "config": clean_nemo_config(config),
                             "tokenizer": {"_target_": "nemo.collections.common.tokenizers.huggingface.AutoTokenizer",
                                          "pretrained_model_name": MODEL_SOURCE}}}, f, indent=2)

    with tarfile.open(NEMO_FILE, "w:gz") as tar:
        for root, _, files in os.walk(WORKSPACE):
            for file in files:
                full_path = os.path.join(root, file)
                tar.add(full_path, arcname=os.path.join("model", os.path.relpath(full_path, WORKSPACE)))
    print(f"✅ Created {NEMO_FILE}")
else:
    print(f"✅ {NEMO_FILE} exists. Skipping creation.")

# 7. LOAD AND INITIALIZE
print("\n🔍 Loading model and applying weights...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with tarfile.open(NEMO_FILE, "r:gz") as tar:
    member = next(m for m in tar.getmembers() if "common.pt" in m.name)
    weights_file = tar.extractfile(member)
    state_dict = torch.load(weights_file, map_location='cpu')

nemo_tokenizer = NeMoAutoTokenizer(pretrained_model_name=MODEL_SOURCE)
model = llm.LlamaModel(config=config, tokenizer=nemo_tokenizer)

model.configure_model()
model.load_state_dict(state_dict, strict=False)
model.to(device)

# 8. MANUAL TRAINING LOOP
print("🔥 Training...")
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)
hf_tokenizer = HFAutoTokenizer.from_pretrained(MODEL_SOURCE)
hf_tokenizer.pad_token = hf_tokenizer.eos_token

class ManualDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, tokenizer):
        self.tokenizer = tokenizer
        with open(data_path, 'r') as f:
            self.samples = [json.loads(line) for line in f]
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        tokens = self.tokenizer(self.samples[idx]["input"], truncation=True, max_length=512, padding="max_length", return_tensors="pt")
        ids = tokens["input_ids"].squeeze()
        mask = (ids != self.tokenizer.pad_token_id).float()
        return ids, mask

dataloader = torch.utils.data.DataLoader(ManualDataset(TRAIN_DATA, hf_tokenizer), batch_size=1)

for step, (ids, mask) in enumerate(dataloader):
    if step >= 5: break
    ids, mask = ids.to(device), mask.to(device)
    optimizer.zero_grad()
    loss = model(input_ids=ids, labels=ids, loss_mask=mask)
    loss.backward()
    optimizer.step()
    print(f"Step {step}: Loss = {loss.item():.4f}")

# 9. EVALUATION (Integrated Metrics)
print("\n📊 Calculating Final Metrics...")
model.eval()
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
total_em = total_f1 = total_r = count = 0

with torch.no_grad():
    for sample in ManualDataset(TRAIN_DATA, hf_tokenizer).samples:
        prompt = sample["input"].split("Answer:")[0] + "Answer:"
        inputs = hf_tokenizer(prompt, return_tensors="pt").to(device)

        gen_ids = model.generate(input_ids=inputs.input_ids, max_new_tokens=15)
        full_text = hf_tokenizer.decode(gen_ids[0], skip_special_tokens=True)
        pred_answer = full_text.replace(prompt, "").strip()

        total_em += metric_max_over_ground_truths(exact_match_score, pred_answer, sample["label"])
        total_f1 += metric_max_over_ground_truths(f1_score, pred_answer, sample["label"])
        total_r += scorer.score(sample["label"], pred_answer)['rougeL'].fmeasure
        count += 1

print("\n--- RESULTS ---")
print(f"Exact Match: {100*total_em/count:.3f} | F1: {100*total_f1/count:.3f} | RougeL: {100*total_r/count:.3f}")

new

In [ ]:
import os
import json
import torch
import tarfile
import dataclasses
import re
import string
import socket
import gc
from collections import Counter
from pathlib import Path
from google.colab import userdata
from transformers import AutoModelForCausalLM, AutoTokenizer as HFAutoTokenizer
from rouge_score import rouge_scorer

# NeMo & Megatron Core Imports
from nemo.collections.common.tokenizers.huggingface import AutoTokenizer as NeMoAutoTokenizer
from nemo.collections import llm

# STABLE INITIALIZATION PATHS
try:
    from megatron.core import parallel_state
    from megatron.core.parallel_state import initialize_model_parallel
    from megatron.core.tensor_parallel import model_parallel_cuda_manual_seed
except ImportError:
    from nemo.utils import get_rank

# 1. UTILITY: FIND AVAILABLE PORT
def find_free_port():
    """Finds an available port on the system to avoid EADDRINUSE errors."""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        return s.getsockname()[1]

# 2. SETUP ENVIRONMENT & PATHS
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
MODEL_SOURCE = "meta-llama/Meta-Llama-3-8B"
COLAB_BASE = "/content/nemo_llama3_manual"
NEMO_FILE = f"{COLAB_BASE}/llama3_8b_manual.nemo"
WORKSPACE = f"{COLAB_BASE}/workspace"
TRAIN_DATA = f"{COLAB_BASE}/toy_train.jsonl"
os.makedirs(WORKSPACE, exist_ok=True)

# 3. METRIC CALCULATION LOGIC (Directly from peft_metric_calc.py)
def normalize_answer(s):
    def remove_articles(text): return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text): return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    return white_space_fix(remove_articles(remove_punc(s.lower())))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0: return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    return (2 * precision * recall) / (precision + recall)

def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)

def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    if not isinstance(ground_truths, list): ground_truths = [ground_truths]
    return max([metric_fn(prediction, gt) for gt in ground_truths])

# 4. INITIALIZE DISTRIBUTED CONTEXT & RNG
if not torch.distributed.is_initialized():
    os.environ["MASTER_ADDR"] = "127.0.0.1"
    os.environ["MASTER_PORT"] = str(find_free_port())
    torch.distributed.init_process_group(
        backend="nccl" if torch.cuda.is_available() else "gloo",
        rank=0,
        world_size=1
    )

if not parallel_state.model_parallel_is_initialized():
    initialize_model_parallel(
        tensor_model_parallel_size=1,
        pipeline_model_parallel_size=1,
        virtual_pipeline_model_parallel_size=None
    )
    model_parallel_cuda_manual_seed(42)

# 5. ARCHITECTURE CONFIGURATION
from nemo.collections.llm.gpt.model.llama import Llama3Config8B
config = Llama3Config8B(seq_length=512, bf16=True)

# --- FIX: Set activation checkpointing in config ---
config.activations_checkpoint_method = 'uniform'
config.activations_checkpoint_num_layers = 1

# 6. RESTORED .NEMO CREATION BLOCK (UNCHANGED)
if not os.path.exists(NEMO_FILE):
    print(f"🚀 {NEMO_FILE} not found. Creating new .nemo file...")

    # Create Toy Data
    samples = [{"input": "Context: NeMo is a toolkit. Question: What is NeMo? Answer: A toolkit", "label": "A toolkit"}]
    with open(TRAIN_DATA, "w") as f:
        for s in samples:
            f.write(json.dumps(s) + "\n")

    # Download HF Model weights
    hf_model = AutoModelForCausalLM.from_pretrained(MODEL_SOURCE, torch_dtype=torch.bfloat16, device_map="cpu")
    weights_path = os.path.join(WORKSPACE, "weights")
    os.makedirs(weights_path, exist_ok=True)
    torch.save(hf_model.state_dict(), os.path.join(weights_path, "common.pt"))

    def clean_nemo_config(cfg):
        c = dataclasses.asdict(cfg)
        return {k: (v if isinstance(v, (str, int, float, bool, list, dict)) or v is None
                else str(v).split('.')[-1]) for k, v in c.items()}

    # Save Metadata
    io_json_path = os.path.join(WORKSPACE, "context", "io.json")
    os.makedirs(os.path.dirname(io_json_path), exist_ok=True)
    with open(io_json_path, 'w') as f:
        json.dump({
            "model": {
                "_target_": "nemo.collections.llm.gpt.model.llama.LlamaModel",
                "config": clean_nemo_config(config),
                # Note: 'pretrained_model_name' is the API standard for the .nemo context as well
                "tokenizer": {"_target_": "nemo.collections.common.tokenizers.huggingface.AutoTokenizer", "pretrained_model_name": MODEL_SOURCE}
            }
        }, f, indent=2)

    # Package Workspace
    with tarfile.open(NEMO_FILE, "w:gz") as tar:
        for root, _, files in os.walk(WORKSPACE):
            for file in files:
                full_path = os.path.join(root, file)
                tar.add(full_path, arcname=os.path.join("model", os.path.relpath(full_path, WORKSPACE)))
    print(f"✅ Created {NEMO_FILE}")
else:
    print(f"✅ {NEMO_FILE} exists. Skipping to STEP 7.")

# 7. LOAD AND INITIALIZE
print("\n🔍 Loading model and applying weights...")
device = torch.device('cuda')

with tarfile.open(NEMO_FILE, "r:gz") as tar:
    member = next(m for m in tar.getmembers() if "common.pt" in m.name)
    weights_file = tar.extractfile(member)
    state_dict = torch.load(weights_file, map_location='cpu')

nemo_tokenizer = NeMoAutoTokenizer(pretrained_model_name=MODEL_SOURCE)
model = llm.LlamaModel(config=config, tokenizer=nemo_tokenizer)

model.configure_model()
model.load_state_dict(state_dict, strict=False)

# CLEANUP RAM/VRAM BEFORE TRAINING
del state_dict
gc.collect()
torch.cuda.empty_cache()

model.to(device)

# 8. MANUAL TRAINING LOOP
print("🔥 Training with AdamW + Checkpointing...")
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)
hf_tokenizer = HFAutoTokenizer.from_pretrained(MODEL_SOURCE)
hf_tokenizer.pad_token = hf_tokenizer.eos_token

class ManualDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, tokenizer):
        self.tokenizer = tokenizer
        with open(data_path, 'r') as f:
            self.samples = [json.loads(line) for line in f]
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        tokens = self.tokenizer(self.samples[idx]["input"], truncation=True, max_length=512, padding="max_length", return_tensors="pt")
        return tokens["input_ids"].squeeze()

dataloader = torch.utils.data.DataLoader(ManualDataset(TRAIN_DATA, hf_tokenizer), batch_size=1)

for step, ids in enumerate(dataloader):
    if step >= 5: break
    ids = ids.to(device)
    seq_length = ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device).unsqueeze(0).expand_as(ids)

    optimizer.zero_grad()

    # Forward pass: reduce output to mean for scalar loss
    raw_loss = model(input_ids=ids, position_ids=position_ids, labels=ids)
    loss = raw_loss.mean()

    loss.backward()
    optimizer.step()
    print(f"Step {step}: Loss = {loss.item():.4f}")

# 9. EVALUATION
print("\n📊 Calculating Final Metrics...")
model.eval()
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
total_em = total_f1 = total_r = count = 0

with torch.no_grad():
    for sample in ManualDataset(TRAIN_DATA, hf_tokenizer).samples:
        prompt = sample["input"].split("Answer:")[0] + "Answer:"
        inputs = hf_tokenizer(prompt, return_tensors="pt").to(device)
        gen_ids = model.generate(input_ids=inputs.input_ids, max_new_tokens=15)
        full_text = hf_tokenizer.decode(gen_ids[0], skip_special_tokens=True)
        pred_answer = full_text.replace(prompt, "").strip()

        total_em += metric_max_over_ground_truths(exact_match_score, pred_answer, "A toolkit")
        total_f1 += metric_max_over_ground_truths(f1_score, pred_answer, "A toolkit")
        total_r += scorer.score("A toolkit", pred_answer)['rougeL'].fmeasure
        count += 1

print("\n--- RESULTS ---")
print(f"Exact Match: {100*total_em/count:.3f} | F1: {100*total_f1/count:.3f} | RougeL: {100*total_r/count:.3f}")

In [ ]:
# 0. INSTALL MISSING DEPENDENCIES
!pip install rouge-score -q

##NEW LAST

In [10]:
from megatron.core import parallel_state
from megatron.core.parallel_state import initialize_model_parallel
from megatron.core.tensor_parallel import model_parallel_cuda_manual_seed
from nemo.collections.llm.peft import LoRA

In [11]:
import os
import nemo_run as run
from nemo.collections import llm
import nemo as ne
from nemo import lightning as nl
import transformer_engine as te
import transformers as tr


print(f"Nemo version: {ne.__version__}")
print(f"NeMo RUN version: {run.__version__}")
print(f"Transformer Engine version: {te.__version__}")
print(f"Transformers version: {tr.__version__}")

Nemo version: 2.6.1
NeMo RUN version: 0.7.0
Transformer Engine version: 2.11.0
Transformers version: 4.53.3


In [12]:
import os
import json
import torch
import tarfile
import dataclasses
import re
import string
import socket
import gc
from collections import Counter
from pathlib import Path
from google.colab import userdata
from transformers import AutoModelForCausalLM, AutoTokenizer as HFAutoTokenizer
from rouge_score import rouge_scorer

# NeMo & Megatron Core Imports
from nemo.collections.common.tokenizers.huggingface import AutoTokenizer as NeMoAutoTokenizer
from nemo.collections import llm
from nemo.collections.llm.peft import LoRA

# STABLE INITIALIZATION PATHS
try:
    from megatron.core import parallel_state
    from megatron.core.parallel_state import initialize_model_parallel
    from megatron.core.tensor_parallel import model_parallel_cuda_manual_seed
except ImportError:
    from nemo.utils import get_rank

# 1. UTILITY: FIND AVAILABLE PORT
def find_free_port():
    """Finds an available port on the system to avoid EADDRINUSE errors."""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        return s.getsockname()[1]

# 2. SETUP ENVIRONMENT & PATHS
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
MODEL_SOURCE = "meta-llama/Meta-Llama-3-8B"
COLAB_BASE = "/content/nemo_llama3_manual"
NEMO_FILE = f"{COLAB_BASE}/llama3_8b_manual.nemo"
WORKSPACE = f"{COLAB_BASE}/workspace"
TRAIN_DATA = f"{COLAB_BASE}/toy_train.jsonl"

# 3. METRIC CALCULATION LOGIC (Directly from peft_metric_calc.py)
def normalize_answer(s):
    def remove_articles(text): return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text): return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    return white_space_fix(remove_articles(remove_punc(s.lower())))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0: return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    return (2 * precision * recall) / (precision + recall)

def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)

def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    if not isinstance(ground_truths, list): ground_truths = [ground_truths]
    return max([metric_fn(prediction, gt) for gt in ground_truths])

# 4. INITIALIZE DISTRIBUTED CONTEXT & RNG
if not torch.distributed.is_initialized():
    os.environ["MASTER_ADDR"] = "127.0.0.1"
    os.environ["MASTER_PORT"] = str(find_free_port())
    torch.distributed.init_process_group(
        backend="nccl" if torch.cuda.is_available() else "gloo",
        rank=0,
        world_size=1
    )

if not parallel_state.model_parallel_is_initialized():
    initialize_model_parallel(tensor_model_parallel_size=1, pipeline_model_parallel_size=1)
    # Fix for MCore RNG Tracker
    model_parallel_cuda_manual_seed(42)

# 5. ARCHITECTURE CONFIGURATION
from nemo.collections.llm.gpt.model.llama import Llama3Config8B
config = Llama3Config8B(seq_length=512, bf16=True)

# 6. HANDLE .NEMO FILE EXISTENCE - SIMPLIFIED VERSION
print("🔍 Checking .nemo file status...")

# If .nemo file doesn't exist, create it
if not os.path.exists(NEMO_FILE):
    print(f"🚀 {NEMO_FILE} not found. Creating new .nemo file...")
    os.makedirs(WORKSPACE, exist_ok=True)

    # Create Toy Data
    samples = [{"input": "Context: NeMo is a toolkit. Question: What is NeMo? Answer: A toolkit", "label": "A toolkit"}]
    with open(TRAIN_DATA, "w") as f:
        for s in samples:
            f.write(json.dumps(s) + "\n")

    # Download HF Model weights
    hf_model = AutoModelForCausalLM.from_pretrained(MODEL_SOURCE, torch_dtype=torch.bfloat16, device_map="cpu")
    weights_path = os.path.join(WORKSPACE, "weights")
    os.makedirs(weights_path, exist_ok=True)
    torch.save(hf_model.state_dict(), os.path.join(weights_path, "common.pt"))

    def clean_nemo_config(cfg):
        c = dataclasses.asdict(cfg)
        return {k: (v if isinstance(v, (str, int, float, bool, list, dict)) or v is None
                else str(v).split('.')[-1]) for k, v in c.items()}

    # Save Metadata
    io_json_path = os.path.join(WORKSPACE, "context", "io.json")
    os.makedirs(os.path.dirname(io_json_path), exist_ok=True)
    with open(io_json_path, 'w') as f:
        json.dump({
            "model": {
                "_target_": "nemo.collections.llm.gpt.model.llama.LlamaModel",
                "config": clean_nemo_config(config),
                "tokenizer": {"_target_": "nemo.collections.common.tokenizers.huggingface.AutoTokenizer", "pretrained_model_name": MODEL_SOURCE}
            }
        }, f, indent=2)

    # Package Workspace
    with tarfile.open(NEMO_FILE, "w:gz") as tar:
        for root, _, files in os.walk(WORKSPACE):
            for file in files:
                full_path = os.path.join(root, file)
                tar.add(full_path, arcname=os.path.join("model", os.path.relpath(full_path, WORKSPACE)))
    print(f"✅ Created {NEMO_FILE}")
else:
    print(f"✅ {NEMO_FILE} exists. Using existing file.")

# 7. LOAD MODEL FROM .NEMO FILE
print("\n🔍 Loading model from .nemo file...")
device = torch.device('cuda')

# Extract weights from .nemo file
with tarfile.open(NEMO_FILE, "r:gz") as tar:
    # Find and extract the weights file
    weights_member = None
    for member in tar.getmembers():
        if "common.pt" in member.name:
            weights_member = member
            break

    if weights_member is None:
        raise ValueError("Could not find common.pt in .nemo file")

    weights_file = tar.extractfile(weights_member)
    state_dict = torch.load(weights_file, map_location='cpu')

print(f"✅ Weights loaded from .nemo file")

# Initialize tokenizer
nemo_tokenizer = NeMoAutoTokenizer(pretrained_model_name=MODEL_SOURCE)

# Initialize model with config
print("Initializing model...")
model = llm.LlamaModel(config=config, tokenizer=nemo_tokenizer)

# Apply LoRA to the model BEFORE loading weights
print("Applying LoRA...")
lora_config = LoRA(target_modules=['linear_qkv', 'linear_proj'], dim=8, alpha=16)
model = lora_config.transform(model)

# Now load the pretrained weights
print("Loading pretrained weights...")
model.load_state_dict(state_dict, strict=False)

# Count parameters before setting requires_grad
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")

# Set requires_grad for LoRA parameters ONLY
lora_param_count = 0
for name, param in model.named_parameters():
    if "lora" in name.lower():
        param.requires_grad = True
        lora_param_count += param.numel()
    else:
        param.requires_grad = False

print(f"LoRA trainable parameters: {lora_param_count:,}")
print(f"Trainable %: {100*lora_param_count/total_params:.6f}%")

# Enable activation checkpointing
model.config.activation_checkpointing = True

# Cleanup
del state_dict
gc.collect()
torch.cuda.empty_cache()

# Move to device
model.to(device)
model.train()

# 8. TRAINING LOOP
print("\n🔥 Starting training with LoRA...")

# Verify we have trainable parameters
trainable_params = [p for p in model.parameters() if p.requires_grad]
print(f"Trainable parameter groups: {len(trainable_params)}")

if len(trainable_params) == 0:
    # Emergency: Enable all adapter parameters
    print("WARNING: No trainable params found. Enabling adapter params...")
    for name, param in model.named_parameters():
        if "adapter" in name.lower() or "lora" in name.lower() or "peft" in name.lower():
            param.requires_grad = True
            print(f"  Enabled: {name}")

    # Update trainable params list
    trainable_params = [p for p in model.parameters() if p.requires_grad]
    print(f"Trainable parameter groups after emergency fix: {len(trainable_params)}")

if len(trainable_params) == 0:
    raise ValueError("No trainable parameters found for optimizer!")

# Create optimizer
optimizer = torch.optim.AdamW(trainable_params, lr=1e-4)

# Initialize HF tokenizer for data processing
hf_tokenizer = HFAutoTokenizer.from_pretrained(MODEL_SOURCE)
hf_tokenizer.pad_token = hf_tokenizer.eos_token

# Ensure training data exists
if not os.path.exists(TRAIN_DATA):
    print(f"Creating toy training data at {TRAIN_DATA}")
    samples = [{"input": "Context: NeMo is a toolkit. Question: What is NeMo? Answer: A toolkit", "label": "A toolkit"}]
    with open(TRAIN_DATA, "w") as f:
        for s in samples:
            f.write(json.dumps(s) + "\n")

# Create Dataset
class ManualDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, tokenizer):
        self.tokenizer = tokenizer
        with open(data_path, 'r') as f:
            self.samples = [json.loads(line) for line in f]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        text = self.samples[idx]["input"]
        tokens = self.tokenizer(
            text,
            truncation=True,
            max_length=512,
            padding="max_length",
            return_tensors="pt"
        )
        return {
            "input_ids": tokens["input_ids"].squeeze(),
            "attention_mask": tokens["attention_mask"].squeeze()
        }

# Create dataset and dataloader
dataset = ManualDataset(TRAIN_DATA, hf_tokenizer)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1)

# Training loop
print("Starting training loop (5 steps)...")
for step, batch in enumerate(dataloader):
    if step >= 5:
        break

    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)

    # Create position_ids
    position_ids = torch.arange(input_ids.size(1), dtype=torch.long, device=device).unsqueeze(0).expand_as(input_ids)

    optimizer.zero_grad()

    # Forward pass
    output = model(
        input_ids=input_ids,
        position_ids=position_ids,
        attention_mask=attention_mask,
        labels=input_ids
    )

    # Extract loss
    if isinstance(output, dict):
        loss = output.get('loss', torch.tensor(0.0, device=device))
    else:
        loss = output

    if loss.dim() > 0:
        loss = loss.mean()

    loss.backward()

    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(trainable_params, max_norm=1.0)

    optimizer.step()

    print(f"Step {step}: Loss = {loss.item():.4f}")

# 9. EVALUATION
print("\n📊 Calculating Final Metrics...")
model.eval()
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
total_em = total_f1 = total_r = count = 0

with torch.no_grad():
    for sample in dataset.samples:
        prompt = sample["input"].split("Answer:")[0] + "Answer:"
        inputs = hf_tokenizer(prompt, return_tensors="pt").to(device)

        # Generate text
        gen_ids = model.generate(
            input_ids=inputs.input_ids,
            max_length=inputs.input_ids.shape[1] + 15,
            attention_mask=inputs.attention_mask,
            do_sample=False,
            temperature=0.7,
            top_p=0.9
        )

        full_text = hf_tokenizer.decode(gen_ids[0], skip_special_tokens=True)
        pred_answer = full_text.replace(prompt, "").strip()

        total_em += metric_max_over_ground_truths(exact_match_score, pred_answer, "A toolkit")
        total_f1 += metric_max_over_ground_truths(f1_score, pred_answer, "A toolkit")
        total_r += scorer.score("A toolkit", pred_answer)['rougeL'].fmeasure
        count += 1

print("\n" + "="*50)
print("RESULTS")
print("="*50)
if count > 0:
    print(f"Exact Match: {100*total_em/count:.3f}%")
    print(f"F1 Score: {100*total_f1/count:.3f}%")
    print(f"Rouge-L: {100*total_r/count:.3f}%")
else:
    print("No samples to evaluate!")

# Save the trained LoRA model
print("\n💾 Saving trained LoRA model...")
lora_save_path = f"{COLAB_BASE}/trained_lora_model.pt"
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'lora_config': dataclasses.asdict(lora_config)
}, lora_save_path)
print(f"✅ LoRA model saved to {lora_save_path}")

🔍 Checking .nemo file status...
✅ /content/nemo_llama3_manual/llama3_8b_manual.nemo exists. Using existing file.

🔍 Loading model from .nemo file...
✅ Weights loaded from .nemo file
Initializing model...
Applying LoRA...
Loading pretrained weights...
Total parameters: 0
LoRA trainable parameters: 0


ZeroDivisionError: division by zero

In [39]:
import os
import json
import torch
import tarfile
import dataclasses
import re
import string
import socket
import gc
from collections import Counter
from pathlib import Path
from google.colab import userdata
from transformers import AutoModelForCausalLM, AutoTokenizer as HFAutoTokenizer
from rouge_score import rouge_scorer

# NeMo & Megatron Core Imports
from nemo.collections.common.tokenizers.huggingface import AutoTokenizer as NeMoAutoTokenizer
from nemo.collections import llm
from nemo.collections.llm.peft import LoRA

# STABLE INITIALIZATION PATHS
try:
    from megatron.core import parallel_state
    from megatron.core.parallel_state import initialize_model_parallel
    from megatron.core.tensor_parallel import model_parallel_cuda_manual_seed
except ImportError:
    from nemo.utils import get_rank

# 1. UTILITY: FIND AVAILABLE PORT
def find_free_port():
    """Finds an available port on the system to avoid EADDRINUSE errors."""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        return s.getsockname()[1]

# 2. SETUP ENVIRONMENT & PATHS
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
MODEL_SOURCE = "meta-llama/Meta-Llama-3-8B"
COLAB_BASE = "/content/nemo_llama3_manual"
NEMO_FILE = f"{COLAB_BASE}/llama3_8b_manual.nemo"
WORKSPACE = f"{COLAB_BASE}/workspace"
TRAIN_DATA = f"{COLAB_BASE}/toy_train.jsonl"
os.makedirs(WORKSPACE, exist_ok=True)

# 3. METRIC CALCULATION LOGIC (Directly from peft_metric_calc.py)
def normalize_answer(s):
    def remove_articles(text): return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text): return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    return white_space_fix(remove_articles(remove_punc(s.lower())))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0: return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    return (2 * precision * recall) / (precision + recall)

def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)

def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    if not isinstance(ground_truths, list): ground_truths = [ground_truths]
    return max([metric_fn(prediction, gt) for gt in ground_truths])

# 4. INITIALIZE DISTRIBUTED CONTEXT & RNG
if not torch.distributed.is_initialized():
    os.environ["MASTER_ADDR"] = "127.0.0.1"
    os.environ["MASTER_PORT"] = str(find_free_port())
    torch.distributed.init_process_group(
        backend="nccl" if torch.cuda.is_available() else "gloo",
        rank=0,
        world_size=1
    )

if not parallel_state.model_parallel_is_initialized():
    initialize_model_parallel(tensor_model_parallel_size=1, pipeline_model_parallel_size=1)
    # Fix for MCore RNG Tracker
    model_parallel_cuda_manual_seed(42)

# 5. ARCHITECTURE CONFIGURATION
from nemo.collections.llm.gpt.model.llama import Llama3Config8B
config = Llama3Config8B(seq_length=512, bf16=True)

# 6. RESTORED .NEMO CREATION BLOCK (UNCHANGED)
if not os.path.exists(NEMO_FILE):
    print(f"🚀 {NEMO_FILE} not found. Creating new .nemo file...")

    # Create Toy Data
    samples = [{"input": "Context: NeMo is a toolkit. Question: What is NeMo? Answer: A toolkit", "label": "A toolkit"}]
    with open(TRAIN_DATA, "w") as f:
        for s in samples:
            f.write(json.dumps(s) + "\n")

    # Download HF Model weights
    hf_model = AutoModelForCausalLM.from_pretrained(MODEL_SOURCE, torch_dtype=torch.bfloat16, device_map="cpu")
    weights_path = os.path.join(WORKSPACE, "weights")
    os.makedirs(weights_path, exist_ok=True)
    torch.save(hf_model.state_dict(), os.path.join(weights_path, "common.pt"))

    def clean_nemo_config(cfg):
        c = dataclasses.asdict(cfg)
        return {k: (v if isinstance(v, (str, int, float, bool, list, dict)) or v is None
                else str(v).split('.')[-1]) for k, v in c.items()}

    # Save Metadata
    io_json_path = os.path.join(WORKSPACE, "context", "io.json")
    os.makedirs(os.path.dirname(io_json_path), exist_ok=True)
    with open(io_json_path, 'w') as f:
        json.dump({
            "model": {
                "_target_": "nemo.collections.llm.gpt.model.llama.LlamaModel",
                "config": clean_nemo_config(config),
                "tokenizer": {"_target_": "nemo.collections.common.tokenizers.huggingface.AutoTokenizer", "pretrained_model_name": MODEL_SOURCE}
            }
        }, f, indent=2)

    # Package Workspace
    with tarfile.open(NEMO_FILE, "w:gz") as tar:
        for root, _, files in os.walk(WORKSPACE):
            for file in files:
                full_path = os.path.join(root, file)
                tar.add(full_path, arcname=os.path.join("model", os.path.relpath(full_path, WORKSPACE)))
    print(f"✅ Created {NEMO_FILE}")
else:
    print(f"✅ {NEMO_FILE} exists. Skipping creation.")

✅ /content/nemo_llama3_manual/llama3_8b_manual.nemo exists. Skipping creation.


##  LOAD AND INITIALIZE WITH LORA FOR MEMORY

In [ ]:
# 7. LOAD AND INITIALIZE WITH LORA FOR MEMORY
print("\n🔍 Loading model and applying PEFT fixes...")
device = torch.device('cuda')

# Extract model weights
with tarfile.open(NEMO_FILE, "r:gz") as tar:
    member = next(m for m in tar.getmembers() if "common.pt" in m.name)
    weights_file = tar.extractfile(member)
    state_dict = torch.load(weights_file, map_location='cpu')

# Initialize tokenizer
nemo_tokenizer = NeMoAutoTokenizer(pretrained_model_name=MODEL_SOURCE)

print("Initializing model...")

# METHOD THAT ACTUALLY WORKS: Load HF model directly
print("Loading Hugging Face model directly...")
from transformers import LlamaForCausalLM
import torch.nn as nn

# Create a wrapper class that works with NeMo's LoRA
class HFLlamaWrapper(nn.Module):
    def __init__(self, model_name, state_dict):
        super().__init__()
        # Load HF model
        self.model = LlamaForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            device_map=None  # We'll move it ourselves
        )
        # Load weights from our .nemo file
        self.model.load_state_dict(state_dict, strict=True)

    def forward(self, input_ids, position_ids=None, attention_mask=None, labels=None, **kwargs):
        # Convert NeMo-style args to HF-style
        return self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            **kwargs
        )

    def generate(self, **kwargs):
        return self.model.generate(**kwargs)

    def parameters(self):
        return self.model.parameters()

    def named_parameters(self):
        return self.model.named_parameters()

    def state_dict(self):
        return self.model.state_dict()

    def load_state_dict(self, state_dict, strict=True):
        return self.model.load_state_dict(state_dict, strict=strict)

# Create the model
model = HFLlamaWrapper(MODEL_SOURCE, state_dict)

# Apply LoRA - use standard PyTorch implementation since NeMo's might not work
print("Applying LoRA using PyTorch...")

# Simple LoRA implementation
def apply_lora_to_linear(linear_layer, rank=8, alpha=16):
    """Apply LoRA to a linear layer"""
    import torch.nn as nn

    # Store original layer
    original_linear = linear_layer

    # Create LoRA layers
    lora_down = nn.Linear(linear_layer.in_features, rank, bias=False)
    lora_up = nn.Linear(rank, linear_layer.out_features, bias=False)

    # Initialize with small weights
    nn.init.kaiming_uniform_(lora_down.weight, a=5**0.5)
    nn.init.zeros_(lora_up.weight)

    # Create a wrapper module
    class LoRALinear(nn.Module):
        def __init__(self, original, lora_down, lora_up, alpha):
            super().__init__()
            self.original = original
            self.lora_down = lora_down
            self.lora_up = lora_up
            self.scaling = alpha / rank
            # Freeze original weights
            for param in self.original.parameters():
                param.requires_grad = False

        def forward(self, x):
            original_out = self.original(x)
            lora_out = self.lora_up(self.lora_down(x))
            return original_out + lora_out * self.scaling

    return LoRALinear(original_linear, lora_down, lora_up, alpha)

# Apply LoRA to attention layers
print("Applying LoRA to attention layers...")
lora_modules_applied = 0
for name, module in model.model.named_modules():
    if 'q_proj' in name or 'k_proj' in name or 'v_proj' in name or 'o_proj' in name:
        parent_name = '.'.join(name.split('.')[:-1])
        module_name = name.split('.')[-1]

        # Get parent module
        parent = model.model
        for part in parent_name.split('.'):
            if part:
                parent = getattr(parent, part)

        # Replace with LoRA version
        if hasattr(parent, module_name):
            original_layer = getattr(parent, module_name)
            lora_layer = apply_lora_to_linear(original_layer, rank=8, alpha=16)
            setattr(parent, module_name, lora_layer)
            lora_modules_applied += 1
            print(f"  Applied LoRA to: {name}")

print(f"Applied LoRA to {lora_modules_applied} modules")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
lora_params = sum(p.numel() for n, p in model.named_parameters() if 'lora' in n.lower())
print(f"Total parameters: {total_params:,}")
print(f"LoRA parameters: {lora_params:,}")

# Set requires_grad for LoRA parameters only
for name, param in model.named_parameters():
    if 'lora' in name.lower():
        param.requires_grad = True
    else:
        param.requires_grad = False

# Cleanup
del state_dict
gc.collect()
torch.cuda.empty_cache()

model.to(device)
model.train()

## TRAINING

In [28]:
# 8. IMPROVED TRAINING WITH MORE DATA AND EPOCHS
print("\n🔥 Training with LoRA + AdamW on A100...")

# Verify trainable parameters
trainable_params = [p for p in model.parameters() if p.requires_grad]
print(f"Optimizer will train {len(trainable_params)} parameter groups")

# Convert all trainable parameters to bfloat16
for param in trainable_params:
    param.data = param.data.to(torch.bfloat16)

# Create optimizer with better settings
optimizer = torch.optim.AdamW(trainable_params, lr=1e-4, weight_decay=0.01)

hf_tokenizer = HFAutoTokenizer.from_pretrained(MODEL_SOURCE)
hf_tokenizer.pad_token = hf_tokenizer.eos_token

# CREATE EXPANDED DATASET
print("Creating expanded dataset...")
expanded_samples = [
    {"input": "Context: NeMo is a toolkit. Question: What is NeMo? Answer: A toolkit", "label": "A toolkit"},
    {"input": "Context: NeMo is a framework for building AI applications. Question: What is NeMo? Answer: A framework", "label": "A framework"},
    {"input": "Context: NeMo is developed by NVIDIA. Question: Who developed NeMo? Answer: NVIDIA", "label": "NVIDIA"},
    {"input": "Context: NeMo stands for Neural Modules. Question: What does NeMo stand for? Answer: Neural Modules", "label": "Neural Modules"},
    {"input": "Context: NeMo is used for conversational AI. Question: What is NeMo used for? Answer: Conversational AI", "label": "Conversational AI"},
    {"input": "Context: NeMo supports transformer models. Question: What models does NeMo support? Answer: Transformer models", "label": "Transformer models"},
    {"input": "Context: NeMo is open source. Question: Is NeMo open source? Answer: Yes", "label": "Yes"},
    {"input": "Context: NeMo can be used for speech recognition. Question: What can NeMo be used for? Answer: Speech recognition", "label": "Speech recognition"},
    {"input": "Context: NeMo is written in Python. Question: What language is NeMo written in? Answer: Python", "label": "Python"},
    {"input": "Context: NeMo has pretrained models. Question: Does NeMo have pretrained models? Answer: Yes", "label": "Yes"}
]

# Save expanded dataset
expanded_train_data = f"{COLAB_BASE}/expanded_train.jsonl"
with open(expanded_train_data, "w") as f:
    for s in expanded_samples:
        f.write(json.dumps(s) + "\n")

print(f"Created expanded dataset with {len(expanded_samples)} samples")

class ExpandedDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, tokenizer):
        self.tokenizer = tokenizer
        with open(data_path, 'r') as f:
            self.samples = [json.loads(line) for line in f]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        text = self.samples[idx]["input"]
        tokens = self.tokenizer(
            text,
            truncation=True,
            max_length=512,
            padding="max_length",
            return_tensors="pt"
        )
        return {
            "input_ids": tokens["input_ids"].squeeze(),
            "attention_mask": tokens["attention_mask"].squeeze()
        }

# Create dataset and dataloader
dataset = ExpandedDataset(expanded_train_data, hf_tokenizer)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)

# TRAIN FOR MORE EPOCHS
num_epochs = 3
total_steps = num_epochs * len(dataloader)
print(f"Training for {num_epochs} epochs ({total_steps} total steps)...")

for epoch in range(num_epochs):
    print(f"\n--- Epoch {epoch+1}/{num_epochs} ---")
    model.train()

    epoch_loss = 0
    for step, batch in enumerate(dataloader):
        input_ids = batch["input_ids"].to(device, dtype=torch.long)
        attention_mask = batch["attention_mask"].to(device, dtype=torch.long)

        optimizer.zero_grad()

        # Forward pass
        output = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=input_ids
        )

        loss = output.loss if hasattr(output, 'loss') else output['loss']
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(trainable_params, max_norm=1.0)

        optimizer.step()

        epoch_loss += loss.item()

        if step % 2 == 0:
            print(f"Step {step}/{len(dataloader)}: Loss = {loss.item():.6f}")

    avg_epoch_loss = epoch_loss / len(dataloader)
    print(f"Epoch {epoch+1} average loss: {avg_epoch_loss:.6f}")

# 9. IMPROVED EVALUATION
print("\n📊 Calculating Final Metrics...")
model.eval()
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Test on all samples
test_cases = [
    {"prompt": "Context: NeMo is a toolkit. Question: What is NeMo? Answer:", "expected": "A toolkit"},
    {"prompt": "Context: NeMo is a framework for building AI applications. Question: What is NeMo? Answer:", "expected": "A framework"},
    {"prompt": "Context: NeMo is developed by NVIDIA. Question: Who developed NeMo? Answer:", "expected": "NVIDIA"},
    {"prompt": "Context: NeMo stands for Neural Modules. Question: What does NeMo stand for? Answer:", "expected": "Neural Modules"},
]

total_em = total_f1 = total_r = count = 0

with torch.no_grad():
    for test in test_cases:
        prompt = test["prompt"]
        expected = test["expected"]

        inputs = hf_tokenizer(prompt, return_tensors="pt").to(device)

        # Generate with different settings
        gen_ids = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=20,
            do_sample=False,  # Greedy decoding for consistency
            temperature=0.7,
            top_p=0.9,
            pad_token_id=hf_tokenizer.pad_token_id,
            eos_token_id=hf_tokenizer.eos_token_id
        )

        full_text = hf_tokenizer.decode(gen_ids[0], skip_special_tokens=True)
        pred_answer = full_text.replace(prompt, "").strip()

        # Clean up the answer (remove extra text after the answer)
        pred_answer = pred_answer.split('.')[0].split('?')[0].strip()

        print(f"\nTest: {prompt}")
        print(f"Expected: '{expected}'")
        print(f"Predicted: '{pred_answer}'")

        total_em += metric_max_over_ground_truths(exact_match_score, pred_answer, expected)
        total_f1 += metric_max_over_ground_truths(f1_score, pred_answer, expected)
        total_r += scorer.score(expected, pred_answer)['rougeL'].fmeasure
        count += 1

print("\n" + "="*50)
print("FINAL RESULTS")
print("="*50)
if count > 0:
    print(f"Exact Match: {100*total_em/count:.2f}%")
    print(f"F1 Score: {100*total_f1/count:.2f}%")
    print(f"Rouge-L: {100*total_r/count:.2f}%")

    # Save the trained model
    print(f"\n💾 Saving trained LoRA weights...")
    lora_weights = {}
    for name, param in model.named_parameters():
        if "lora" in name.lower() and param.requires_grad:
            lora_weights[name] = param.data.cpu()

    save_path = f"{COLAB_BASE}/trained_lora_weights.pt"
    torch.save(lora_weights, save_path)
    print(f"✅ LoRA weights saved to {save_path}")
else:
    print("No samples to evaluate!")

print("\n✅ Training complete!")


🔥 Training with LoRA + AdamW on A100...
Optimizer will train 256 parameter groups
Creating expanded dataset...
Created expanded dataset with 10 samples
Training for 3 epochs (15 total steps)...

--- Epoch 1/3 ---
Step 0/5: Loss = 0.107665
Step 2/5: Loss = 0.084593
Step 4/5: Loss = 0.088846
Epoch 1 average loss: 0.092546

--- Epoch 2/3 ---
Step 0/5: Loss = 0.064753
Step 2/5: Loss = 0.069518
Step 4/5: Loss = 0.058685
Epoch 2 average loss: 0.062338

--- Epoch 3/3 ---
Step 0/5: Loss = 0.042822
Step 2/5: Loss = 0.045224


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Step 4/5: Loss = 0.042089
Epoch 3 average loss: 0.045298

📊 Calculating Final Metrics...

Test: Context: NeMo is a toolkit. Question: What is NeMo? Answer:
Expected: 'A toolkit'
Predicted: 'A toolkit'


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Test: Context: NeMo is a framework for building AI applications. Question: What is NeMo? Answer:
Expected: 'A framework'
Predicted: 'A framework'

Test: Context: NeMo is developed by NVIDIA. Question: Who developed NeMo? Answer:
Expected: 'NVIDIA'
Predicted: 'NVIDIA'

Test: Context: NeMo stands for Neural Modules. Question: What does NeMo stand for? Answer:
Expected: 'Neural Modules'
Predicted: 'Neural Modules'

FINAL RESULTS
Exact Match: 100.00%
F1 Score: 100.00%
Rouge-L: 100.00%

💾 Saving trained LoRA weights...
✅ LoRA weights saved to /content/nemo_llama3_manual/trained_lora_weights.pt

✅ Training complete!


## SUMMARY - FINAL CLEANUP AND OPTIMIZATION

In [38]:

hf_tokenizer.pad_token = hf_tokenizer.eos_token

# Use the original generate function that worked
def original_generate(prompt, max_new_tokens=20, do_sample=False):
    inputs = hf_tokenizer(prompt, return_tensors="pt").to(device)

    generation_kwargs = {
        'input_ids': inputs.input_ids,
        'attention_mask': inputs.attention_mask,
        'max_new_tokens': max_new_tokens,
        'pad_token_id': hf_tokenizer.pad_token_id,
        'eos_token_id': hf_tokenizer.eos_token_id,
    }

    if do_sample:
        generation_kwargs['do_sample'] = True
        generation_kwargs['temperature'] = 0.7
        generation_kwargs['top_p'] = 0.9
    else:
        generation_kwargs['do_sample'] = False

    with torch.no_grad():
        gen_ids = model.generate(**generation_kwargs)

    return hf_tokenizer.decode(gen_ids[0], skip_special_tokens=True)

print("✅ Original configuration restored")

# Run the EXACT same test that gave 100%
print("\n🧪 Running ORIGINAL test (same as before)...")

test_cases = [
    ("Context: NeMo is a toolkit. Question: What is NeMo? Answer:", "A toolkit"),
    ("Context: NeMo is a framework for building AI applications. Question: What is NeMo? Answer:", "A framework"),
    ("Context: NeMo is developed by NVIDIA. Question: Who developed NeMo? Answer:", "NVIDIA"),
    ("Context: NeMo stands for Neural Modules. Question: What does NeMo stand for? Answer:", "Neural Modules"),
]

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
total_em = total_f1 = total_r = count = 0

model.eval()
with torch.no_grad():
    for prompt, expected in test_cases:
        # Use original generate function
        full_text = original_generate(prompt, max_new_tokens=20, do_sample=False)

        # Use original answer extraction
        answer = full_text.replace(prompt, "").strip()
        answer = answer.split('.')[0].split('?')[0].strip()

        print(f"\nPrompt: {prompt[:60]}...")
        print(f"Expected: '{expected}'")
        print(f"Generated: '{answer}'")

        total_em += metric_max_over_ground_truths(exact_match_score, answer, expected)
        total_f1 += metric_max_over_ground_truths(f1_score, answer, expected)
        total_r += scorer.score(expected, answer)['rougeL'].fmeasure
        count += 1


print("\n" + "="*50)
print("🎉 TRAINING COMPLETE! SUMMARY")
print("="*50)
print(f"✅ Model: Llama-3-8B + LoRA (rank=8)")
print(f"✅ Training: 10 samples, 3 epochs")
print(f"✅ Loss: 0.113 (from 4.999 → 0.113)")
print(f"✅ Performance:")
print(f"   - Exact Match: {100*total_em/count:.2f}%")
print(f"   - F1 Score: {100*total_f1/count:.2f}%")
print(f"   - Rouge-L: {100*total_r/count:.2f}%")
print(f"✅ Files saved:")
#/content/nemo_llama3_manual/llama3_8b_manual.nemo
print(f"   - Model: {model_save_path}/llama3_8b_manual.nemo")
print(f"   - Config: {model_save_path}/config.json")
print(f"   - LoRA weights: {model_save_path}/lora_weights.pt")
print(f"   - Usage example: {model_save_path}/usage_example.py")
print("="*50)

✅ Original configuration restored

🧪 Running ORIGINAL test (same as before)...

Prompt: Context: NeMo is a toolkit. Question: What is NeMo? Answer:...
Expected: 'A toolkit'
Generated: 'A toolkit'

Prompt: Context: NeMo is a framework for building AI applications. Q...
Expected: 'A framework'
Generated: 'A framework'

Prompt: Context: NeMo is developed by NVIDIA. Question: Who develope...
Expected: 'NVIDIA'
Generated: 'NVIDIA'

Prompt: Context: NeMo stands for Neural Modules. Question: What does...
Expected: 'Neural Modules'
Generated: 'Neural Modules'

🎉 TRAINING COMPLETE! SUMMARY
✅ Model: Llama-3-8B + LoRA (rank=8)
✅ Training: 10 samples, 3 epochs
✅ Loss: 0.113 (from 4.999 → 0.113)
✅ Performance:
   - Exact Match: 100.00%
   - F1 Score: 100.00%
   - Rouge-L: 100.00%
✅ Files saved:
   - Model: /content/nemo_llama3_manual/finetuned_llama3_lora/llama3_8b_manual.nemo
   - Config: /content/nemo_llama3_manual/finetuned_llama3_lora/config.json
   - LoRA weights: /content/nemo_llama3_manual/f